In [1]:
import os
import gradio as gr
import plotly.graph_objects as go
import numpy as np
import geopandas as gp
import pandas as pd
import json
import shapely
from PIL import Image

In [2]:
#importing and processing the data
temp_tot = gp.GeoDataFrame(pd.concat([gp.read_file(f'./geojsons/{filename}') for filename in os.listdir('./geojsons')],ignore_index=True))
temp_tot['Building Name'] = np.arange(len(temp_tot))
temp_tot['Area (m²)'] = round(temp_tot['Area (m²)'])
temp_tot['Number of Stories'] = (temp_tot['Number of Stories']).astype(int)

#In the production version this would be labeled by the artificial intelligence
destroyed = temp_tot[temp_tot['Damage Level']==4]

"""
adding a default image, in the production version this would also be provided by the dataset.
For moral reasons i decided not to fill this with more than one image of destroyed buildings since they are peoples homes.
"""
destroyed = destroyed.assign(image=Image.open('./damaged_house.png'))

In [3]:
#load polygons for the turkish administrative borders
with open('turkey_704.geojson') as file:
    districts_json = json.load(file)
districts = gp.read_file('turkey_704.geojson')

districts['num_buildings'] = 0 
#edit the coordinates because the x and y are flipped in the districts
_coords = destroyed.geometry.apply(lambda geom: shapely.ops.transform(lambda x, y: (y, x), geom))
for n,district in enumerate(districts.geometry):
    #count amount of destroyed buildings in each district
    n_buildings = sum(district.contains(_coords))
    if n_buildings != 0:
        districts.loc[n,'num_buildings'] = n_buildings

In [4]:
layout = go.Layout(autosize=False,width=600,height=600)
fig = go.Figure(layout=layout)

fig.add_trace(go.Scattermapbox(
            name='Destroyed',
            customdata=destroyed[['Occupants','Area (m²)','Construction Type','Building Name']].values.tolist(),
            lat=destroyed['geometry'].x,
            lon=destroyed['geometry'].y,
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=10,
                opacity = 1,
                color = 'red',
            ),
            hovertemplate='<b>ID</b>: %{customdata[3]}<br><b>Expected occupancy</b>: %{customdata[0]}<br><b>Area</b>: %{customdata[1]}m<sup>2</sup><br><b>Contruction material</b>: %{customdata[2]}',
            ),
            )

fig.update_layout(
    mapbox_style="open-street-map",
    hovermode='closest',
    hoverdistance =100,
    mapbox=dict(
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= 37,
            lon=36
        ),
        pitch=5,
        zoom=7
    
    )
)

with gr.Blocks() as regionmap:
    map = gr.Plot(fig,scale=2,container=False)
    regionmap.launch(server_port=7880) 

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


In [5]:
analysis_images = [Image.open(f'./veronique_analysis/{filename}') for filename in os.listdir('./veronique_analysis')]

In [6]:
def get_image(id):
    return(destroyed.loc[int(id)].image.resize((400,400)))
def get_data(id):
    _data = destroyed.loc[id][['Building Name','Occupants','Area (m²)','Construction Type','Number of Stories','Hazardous materials','Construction Year']]
    html = '<b>Occupancy</b>: %i <b>Area</b>: %im<sup>2</sup><br> <b>Primary construction material</b>: %s <b>Number of Stories</b>: %s <br><b>Hazardous materials</b>: %s <b>Construction Year</b>: %s' % (_data[1],_data[2],_data[3],_data[4], _data[5], _data[6])
    return(html)

with gr.Blocks() as information:
    with gr.Tab('Analysis'):
        gr.Gallery(value=analysis_images,columns=1,object_fit='contain',preview=True)
    with gr.Tab('Building information'):
        with gr.Column():
            id = gr.Dropdown(label='Building ID',choices=destroyed['Building Name'].tolist())
            info = gr.HTML(value=('<b>Occupancy</b>: __ <b>Area</b>: __m<sup>2</sup><br> <b>Primary construction material</b>: __ <b>Number of Stories</b>: __ <br><b>Hazardous materials</b>: __ <b>Construction Year</b>: __'))
            image = gr.Image(scale=1)
            id.select(get_image,inputs=[id], outputs=[image])
            id.select(get_data,inputs=[id], outputs=[info])
information.launch(server_port=7881)

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [7]:
worldmap_fig = go.Figure(layout=layout)

worldmap_fig.add_trace(go.Choroplethmapbox(geojson=districts_json, 
                                    locations=np.arange(1,len(districts)+1).tolist(), 
                                    z=districts.num_buildings,
                                    customdata=districts[['name','num_buildings']].values.tolist(),
                                    colorscale=[[0, 'rgb(255,255,255)'],[1, 'rgb(255,0,0)']], zauto=True,
                                    marker_opacity=0.5, 
                                    marker_line_width=1, 
                                    hovertemplate='Province: <a href="http://127.0.0.1:7880" target="map"> %{customdata[0]}</a><br>Num damaged: %{customdata[1]}'))

worldmap_fig.update_layout(
    mapbox_style="open-street-map",
    hovermode='closest',
    mapbox=dict(
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= 37,
            lon=36
        ),
        pitch=5,
        zoom=3
    )
)


with gr.Blocks() as worldmap:
    map = gr.Plot(worldmap_fig)

    
worldmap.launch(server_port=7882)   
#worldmap.launch()

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.
